In [30]:
from dotenv import load_dotenv
from IPython.display import display, HTML
import os
from openai import AzureOpenAI
import json

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_GPT35_DEPLOYMENT_NAME = os.getenv("OPENAI_GPT35_DEPLOYMENT_NAME")


client = AzureOpenAI(
  api_key=OPENAI_API_KEY,  
  api_version="2023-10-01-preview",
  azure_endpoint = OPENAI_DEPLOYMENT_ENDPOINT
)

In [31]:
import inspect

# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

In [32]:
import pytz
from datetime import datetime


def get_current_time(location):
    try:
        # Get the timezone for the city
        timezone = pytz.timezone(location)

        # Get the current time in the timezone
        now = datetime.now(timezone)
        current_time = now.strftime("%I:%M:%S %p")

        return current_time
    except:
        return "Sorry, I couldn't find the timezone for that location."

In [33]:
import pandas as pd
import json


def get_stock_market_data(index):
    available_indices = [
        "S&P 500",
        "NASDAQ Composite",
        "Dow Jones Industrial Average",
        "Financial Times Stock Exchange 100 Index",
    ]

    if index not in available_indices:
        return "Invalid index. Please choose from 'S&P 500', 'NASDAQ Composite', 'Dow Jones Industrial Average', 'Financial Times Stock Exchange 100 Index'."

    # Read the CSV file
    data = pd.read_csv("stock_data.csv")

    # Filter data for the given index
    data_filtered = data[data["Index"] == index]

    # Remove 'Index' column
    data_filtered = data_filtered.drop(columns=["Index"])

    # Convert the DataFrame into a dictionary
    hist_dict = data_filtered.to_dict()

    for key, value_dict in hist_dict.items():
        hist_dict[key] = {k: v for k, v in value_dict.items()}

    return json.dumps(hist_dict)

In [34]:
import math
def calculator(num1, num2, operator):
    if operator == "+":
        return str(num1 + num2)
    elif operator == "-":
        return str(num1 - num2)
    elif operator == "*":
        return str(num1 * num2)
    elif operator == "/":
        return str(num1 / num2)
    elif operator == "**":
        return str(num1**num2)
    elif operator == "sqrt":
        return str(math.sqrt(num1))
    else:
        return "Invalid operator"

In [35]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Get the current time in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location name. The pytz is used to get the timezone for that location. Location names should be in a format like America/New_York, Asia/Bangkok, Europe/London",
                    }
                },
                "required": ["location"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_stock_market_data",
            "description": "Get the stock market data for a given index",
            "parameters": {
                "type": "object",
                "properties": {
                    "index": {
                        "type": "string",
                        "enum": [
                            "S&P 500",
                            "NASDAQ Composite",
                            "Dow Jones Industrial Average",
                            "Financial Times Stock Exchange 100 Index",
                        ],
                    },
                },
                "required": ["index"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "calculator",
            "description": "A simple calculator used to perform basic arithmetic operations",
            "parameters": {
                "type": "object",
                "properties": {
                    "num1": {"type": "number"},
                    "num2": {"type": "number"},
                    "operator": {
                        "type": "string",
                        "enum": ["+", "-", "*", "/", "**", "sqrt"],
                    },
                },
                "required": ["num1", "num2", "operator"],
            },
        },
    },
]

available_functions = {
    "get_current_time": get_current_time,
    "get_stock_market_data": get_stock_market_data,
    "calculator": calculator,
}

In [36]:
def run_conversation(messages, tools, available_functions):
    # Step 1: send the conversation and available functions to GPT
    response = client.chat.completions.create(
        model=OPENAI_GPT35_DEPLOYMENT_NAME,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

    response_message = response.choices[0].message

    # Step 2: check if GPT wanted to call a function
    if response_message.tool_calls:
        print("Recommended Function call:")
        print(response_message.tool_calls[0])
        print()

        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors

        function_name = response_message.tool_calls[0].function.name

        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name]

        # verify function has correct number of arguments
        function_args = json.loads(response_message.tool_calls[0].function.arguments)
        if check_args(function_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        # call the function
        function_response = function_to_call(**function_args)
        print("Output of function call:")
        print(function_response)
        print()

        # Step 4: send the info on the function call and function response to GPT

        # adding assistant response to messages
        messages.append(
            {
                "role": response_message.role,
                "function_call": {
                    "name": response_message.tool_calls[0].function.name,
                    "arguments": response_message.tool_calls[0].function.arguments,
                },
                "content": None,
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("Messages in second request:")
        for message in messages:
            print(message)
        print()

        second_response = client.chat.completions.create(
            messages=messages,
            model=OPENAI_GPT35_DEPLOYMENT_NAME,
        )  # get a new response from GPT where it can see the function response

        return second_response

In [38]:
messages = [{"role": "user", "content": "What time is it in New York?"}]
assistant_response = run_conversation(messages, tools, available_functions)
print(assistant_response.choices[0].message.content)

Recommended Function call:
ChatCompletionMessageToolCall(id='call_wmikNhI5PCz7p5hahxoz9M9V', function=Function(arguments='{\n"location": "America/New_York"\n}', name='get_current_time'), type='function')

Output of function call:
09:49:28 AM

Messages in second request:
{'role': 'user', 'content': 'What time is it in New York?'}
{'role': 'assistant', 'function_call': {'name': 'get_current_time', 'arguments': '{\n"location": "America/New_York"\n}'}, 'content': None}
{'role': 'function', 'name': 'get_current_time', 'content': '09:49:28 AM'}

It is currently 09:49 AM in New York.
